# Collaborative filtering recommender system

## Dataset description

In this assignment you will use Apache Spark to build simple movie recommendation system on big MovieLens 20M dataset (https://grouplens.org/datasets/movielens/). The dataset contains 20 million ratings for 27000 movies given by 138000 users. 

Dataset format: comma-separated values (https://en.wikipedia.org/wiki/CSV)

The first line is a header:
userId,movieId,rating,timestamp

_userId_, _movieId_ are integers representing user and movies identifiers correspondingly

_rating_ is a floating point number from 1.0 to 5.0

_timestamp_ is an integer but it won’t be used in the assignment :)

## Reading dataset

Init spark session

In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder \
    .enableHiveSupport() \
    .appName("recSys")\
    .master("local[8]")\
    .getOrCreate()
sc = spark_session.sparkContext

In [2]:
data = spark_session.read.load("/data/movielens/ratings_100k.csv", format="csv", inferSchema=True, header=True)

In this assignment you will use collaborative filtering approach for making predictions. To find latent vector representation for users and items we suggest you to use explicit ALS method. Refer to the Spark MLLib documentation for Python API details: https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html

Split dataset into three folds: on each iteration one fold will be used for testing and the other two folds will be used for training. Wrap necessary values with _Rating_ class.

In [3]:
from pyspark.mllib.recommendation import Rating
import math

fold_count = 3

def to_rating(row):
    return Rating(row["userId"], row["movieId"], row["rating"])

In [4]:
df1, df2 = data.randomSplit([0.9, 0.1], 123)

rdd1 = df1.rdd.map(to_rating)
rdd2 = df2.rdd.map(to_rating)
rdd_all = data.rdd.map(to_rating)

Recommendations: to make your solution more efficient prepare RDDs for all the folds beforehand running the training process.

## Training ALS model

Now it is time to choose training parameters. It is recommended to set the rank equal to 20,
regularization term lambda equal to 0.01 and the number of iterations equal to 5.
Feel free to experiment and choose your own parameters to see how they influence the final score.

In [5]:
rank = 20
iterations = 5
lambda_ = 0.01

Train the explicit ALS model on two of three folds and evaluate its performance on remaining test fold. Performance evaluation should be done using the classic RMSE metric.

In [6]:
scores = []

In [7]:
from pyspark.mllib.recommendation import ALS

In [8]:
rdd_test = rdd2.map(lambda r : (r[0], r[1]))    
model = ALS.train(rdd1, rank = 8, iterations=5, lambda_=0.01, seed = 123)
prediction = model.predictAll(rdd_test).map(lambda x:((x.user, x.product), x.rating))
actual_ratings = rdd2.map(lambda x:((x.user, x.product), x.rating))
joined_data = actual_ratings.join(prediction)
score = math.sqrt(joined_data.map(lambda x: (x[1][0] - x[1][1]) ** 2).mean())
scores.append(score)

Average RMSE scores on all the three folds and output the result to stdout.
You could refer to publicly available benchmarks (e.g. http://mymedialite.net/examples/datasets.html)
to find out what score to expect.

In [9]:
score = sum(scores) / len(scores)

In [10]:
score

1.0653847782818984